In [1]:
from tensorflow import keras, expand_dims
import numpy as np
model_name = 'VisFin_v5.h5'

In [2]:
def make_new_model(dropout = 0):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18),padding='same', input_shape=(201, 43, 1), activation='elu'))
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18), padding='same', activation='elu'))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Conv2D(9, kernel_size = (6,12),padding='same', activation='elu'))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(84, activation='elu'))
    model.add(keras.layers.Dense(42, activation='elu'))
    model.add(keras.layers.Dense(7,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [3]:
from modules.data_feed import *
modeling = PrepareData('daily__us__nasdaq_stocks.csv', '<DATE>', skip=5)
modeling.clean_db()

In [4]:
try:
    VisCV = keras.models.load_model(model_name)
    print('Loading!')
except OSError:
    VisCV = make_new_model(dropout=0.3)
    print('Compiling!')

Loading!


In [5]:
# # TRAINING
# data = modeling.mass_hacking(tick=1, horizon=84)
# tr_data = expand_dims(np.array(data[0]),axis=-1)
# tr_key = np.array(data[1])
# if tr_data.shape[0] > 0:
#     VisCV.fit(tr_data, tr_key, epochs=3, verbose=2)
VisCV.save(model_name)


In [6]:
# EVALUATE
data = modeling.mass_hacking(tick=0, horizon=84)
tr_data = expand_dims(np.array(data[0]),axis=-1)
tr_key = np.array(data[1])
eval_ = VisCV.evaluate(tr_data,tr_key, verbose=1)
print(model_name+': ',eval_[0], eval_[1])

67/67 [==============================] - 7s 55ms/step - loss: 0.8271 - accuracy: 0.7608
VisCV_v5.h5:  0.8270650506019592 0.7608286142349243


In [7]:
# # # convert to JS
# import tensorflowjs
# # tensorflowjs.converters.save_keras_model(VisFin0, 'js_format\\VisFin_0')
# # tensorflowjs.converters.save_keras_model(VisFin1, 'js_format\\VisFin_1')
# # tensorflowjs.converters.save_keras_model(VisFin2, 'js_format\\VisFin_2')
# # tensorflowjs.converters.save_keras_model(VisFin3, 'js_format\\VisFin_3')
